In [1]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast


In [2]:
def load_and_preprocess_data(path):
    df = pd.read_csv(path)
    print("Dataset Preview:")
    print(df.head(), "\n")
    df.drop(columns=['link', 'source', 'NER', 'site'], inplace=True, errors='ignore')
    # Lowercase the ingredients while retaining their list format (as string)
    df['ingredients'] = df['ingredients'].str.lower()
    return df

In [3]:
def build_tfidf_matrix(df):
    vectorizer = TfidfVectorizer(stop_words='english')
    matrix = vectorizer.fit_transform(df['ingredients'])
    return vectorizer, matrix


In [4]:
def recommend_recipe(user_ingredients, vectorizer, matrix, df, top_n=1):
    user_input = user_ingredients.lower().replace(',', ' ')
    user_vector = vectorizer.transform([user_input])
    scores = cosine_similarity(user_vector, matrix).flatten()
    if scores.max() == 0:
        return []
    best_indices = scores.argsort()[::-1][:top_n]
    recommendations = []
    for idx in best_indices:
        recommendations.append({
            'title': df.iloc[idx]['title'],
            'ingredients': df.iloc[idx]['ingredients'],
            'directions': df.iloc[idx]['directions'],
            'similarity': scores[idx]
        })
    return recommendations

In [5]:
def print_list(text):
    # If text starts with '[' assume it's a list in string form and convert it
    if text.strip().startswith('['):
        items = ast.literal_eval(text)
        for item in items:
            print(f" - {item}")
    else:
        # Otherwise split on commas and print each item
        for item in text.split(','):
            print(f" - {item.strip()}")

In [6]:
def display_recommendations(recommendations):
    if not recommendations:
        print("\nNo matching recipes found. Please try different ingredients.\n")
        return
    print("\n" + "=" * 60)
    for i, rec in enumerate(recommendations, start=1):
        print(f"Recommendation #{i}")
        print(f"Recipe Title     : {rec['title']}")
        print(f"Similarity Score : {rec['similarity']:.2f}")
        print("\nIngredients:")
        print_list(rec['ingredients'])
        print("\nDirections:")
        print_list(rec['directions'])
        print("=" * 60 + "\n")

In [7]:
def main():
    path = 'recipes_data.csv/recipes_data.csv'
    df = load_and_preprocess_data(path)
    vectorizer, matrix = build_tfidf_matrix(df)
    user_input = input("Enter available ingredients (comma separated): ")
    top_n_input = input("How many recommendations would you like? (default 1): ")
    top_n = int(top_n_input) if top_n_input.strip() else 1
    recs = recommend_recipe(user_input, vectorizer, matrix, df, top_n=top_n)
    display_recommendations(recs)

In [8]:
if __name__ == '__main__':
    main()


Dataset Preview:
                   title                                        ingredients  \
0    No-Bake Nut Cookies  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  Jewell Ball'S Chicken  ["1 small jar chipped beef, cut up", "4 boned ...   
2            Creamy Corn  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3          Chicken Funny  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4   Reeses Cups(Candy)    ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  \
0  ["In a heavy 2-quart saucepan, mix brown sugar...   
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                              link    source  \
0   www.cookbooks.com/Recipe-Details.aspx?id=44874  Gathered   
1  www.cookbooks.com/Recipe-Details.aspx?id